In [50]:
import fitz  # PyMuPDF
import os
from dotenv import load_dotenv
from apify_client import ApifyClient

load_dotenv()

True

In [51]:
HUGGING_FACE_API_TOKEN = os.getenv('HUGGING_FACE_API_TOKEN')
GROQ_API_TOKEN = os.getenv('GROQ_API_TOKEN')
GOOGLE_API_KEY = os.getenv('GOOGLE_API_KEY')
# print(HUGGING_FACE_API_TOKEN)

### Define the LLM

In [52]:
# Load LLM Model
# from langchain_huggingface import HuggingFaceEndpoint
# from langchain_groq import ChatGroq
from langchain_google_genai import ChatGoogleGenerativeAI
# from langchain.chains import LLMChain
from langchain_core.prompts import PromptTemplate

question = "Who won the FIFA World Cup in the year 2010? "

template = """Question: {question}

Answer: Let's think step by step."""

prompt = PromptTemplate.from_template(template)

gemini_llm = ChatGoogleGenerativeAI(
    model="models/gemini-2.0-flash",
    temperature=0.7,
    max_tokens=500)

# repo_id = "meta-llama/Llama-3.3-70B-Instruct"

# llm = ChatGroq(
#     model="llama-3.1-8b-instant",
#     temperature=0.0,
#     max_retries=2,
#     api_key=GROQ_API_TOKEN
# )

# llama_llm = HuggingFaceEndpoint(
#     repo_id=repo_id,
#     max_length=500,
#     temperature=0.5,
#     huggingfacehub_api_token=HUGGING_FACE_API_TOKEN,
#     task='text-generation'
# )
# llm_chain = LLMChain(llm=gemini_llm, prompt=prompt)
llm_chain = prompt | gemini_llm
print(llm_chain.invoke({"question": question}))

content="Okay, I'm thinking step by step. The 2010 FIFA World Cup was held in South Africa. I need to recall which team won that year. \n\n... Spain won the 2010 FIFA World Cup." additional_kwargs={} response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-2.0-flash', 'safety_ratings': []} id='run--1984a937-b8e9-42fc-83e2-5915280c63c4-0' usage_metadata={'input_tokens': 29, 'output_tokens': 51, 'total_tokens': 80, 'input_token_details': {'cache_read': 0}}


In [53]:
# def llm_define(llm):
#     return llm

# lama = llm_define(gemini_llm)

# llm_chain = LLMChain(llm=lama, prompt=prompt)
# print(llm_chain.invoke({"question": question}))

### Load PDF Resume

In [55]:
file_path = r"C:\Users\GD\OneDrive\Desktop\Resume\Dilshan-Upasena.pdf"

# Open the PDF
doc = fitz.open(file_path)

# Example: print the first page text
# print(doc[0].get_text())
resume_text = doc[0].get_text()
# print(resume_text)
doc[0].get_text()

'Dilshan Upasena\ng.dilshanupasena@gmail.com\n \n+94723308678\n \nAnuradhapura, Sri Lanka\n \nlinkedin.com/in/dilshan-upasena-98gdu\n \ngithub.com/G-Dilshan\n \nPROFILE\nComputer Science undergraduate with a strong foundation in Machine Learning, Deep Learning, and\nData Science. Skilled in developing Al-powered solutions for real-world challenges, with hands-on\nexperience in computer vision, supervised and unsupervised learning. Familiar with Python and\nPyTorch, with practical experience in cloud deployment (AWS, Docker), API development using Flask\nand FastAPI, and database systems including MySQL, MongoDB, and PostgreSQL. Actively exploring\nGenerative Al, including Retrieval-Augmented Generation (RAG), LangChain, and vector databases such\nas Pinecone and ChromaDB. Familiar with model deployment and experimentation using platforms like\nHugging Face and Ollama, and working with large language models such as LLaMA and Gemini. Deeply\npassionate about academic research and innovat

In [56]:
# Extract text from PDF
def extract_text_from_pdf(uploaded_file):
    # doc = fitz.open(stream=uploaded_file.read(), filetype='pdf')
    doc = fitz.open(file_path)
    text = ''
    for page in doc:
        text += page.get_text()
    return text

In [57]:
resume_text = extract_text_from_pdf(file_path)

In [58]:
# # Extract text from PDF
# def extract_text_from_pdf(uploaded_file):
#     doc = fitz.open(uploaded_file, filetype='pdf')
#     text = ''
#     for page in doc:
#         text += page.get_text()
#     return text

In [59]:
# Ask gemini to generate output
def ask_gemini(prompt, resume_text):
    llm_chain = prompt | gemini_llm
    response = llm_chain.invoke(resume_text)
    if isinstance(response, dict) and 'choices' in response:
        return response['choices'][0]['message']['content']
    return response

In [60]:
prompt_summary = PromptTemplate(
    input_variables=["resume_text"],
    template="Summarize this resume highlighting skills, education, and experience:\n\n{resume_text}"
)

In [61]:
summary = ask_gemini(prompt=prompt_summary, resume_text=resume_text)

In [62]:
summary.content

"Here's a summary of Dilshan Upasena's resume:\n\n**Summary:**\n\nDilshan Upasena is a Computer Science undergraduate with a strong foundation and passion for Machine Learning, Deep Learning, and Data Science, particularly in Generative AI. He possesses hands-on experience in developing AI-powered solutions using Python, PyTorch, and various frameworks for tasks like computer vision, classification, and prediction. He's familiar with cloud deployment, API development, and database management, and is actively exploring cutting-edge technologies like Retrieval-Augmented Generation (RAG), LangChain, and large language models.\n\n**Skills:**\n\n*   **Machine Learning/Data Science:** Supervised/Unsupervised Learning, Computer Vision, Data Visualization, ANN, CNN\n*   **Generative AI & NLP:** LangChain, Hugging Face, Ollama, RAG, LLAMA, Gemini\n*   **Programming Languages:** Python, Java\n*   **Python Packages/Frameworks:** Pytorch, NumPy, Pandas, Scikit-Learn, Matplotlib, Pillow\n*   **Vect

### Get Keywords for Job Search from linkedin

In [63]:
prompt_roadmap = PromptTemplate(
        input_variables=["summary"],  
        template="Based on this resume summary, suggest the best job titles/keywords for searching jobs. Give a comma-separated list only, no explanation.\n\nSummary:\n{summary}"
        )
keywords = ask_gemini(prompt=prompt_roadmap, resume_text={"summary": summary.content})
search_keywords_clean = keywords.content.replace("\n", "").strip()

In [64]:
search_keywords_clean

'Machine Learning Engineer, Data Scientist, AI Engineer, Generative AI Engineer, Deep Learning Engineer, Computer Vision Engineer, NLP Engineer, AI Developer, Machine Learning Developer, Data Science Intern, AI Intern, Python Developer, Pytorch Engineer, RAG Engineer, LangChain Developer, LLM Engineer, AI Research Assistant'

### Define Apify linkedin Scraper

In [54]:
# Load Apify api
APIFY_API_TOKEN = os.getenv('APIFY_API_TOKEN')
apify_client = ApifyClient(token=APIFY_API_TOKEN)


In [65]:
# Fetch Linkedin jobs using Apify
def fetch_linkedin_jobs(search_query, location="Sri Lanka", rows=60):
    run_input = {
        'title': search_query,
        'location': location,
        'rows': rows,
        'proxy': {
            'useApifyProxy': True,
            'apifyProxyGroups': ['RESIDENTIAL']
        }
    }
    run = apify_client.actor('api_key_for_apify').call(run_input=run_input)
    jobs = list(apify_client.dataset(run['defaultDatasetId']).iterate_items())
    return jobs

In [66]:
linkedin_results = fetch_linkedin_jobs(search_query=search_keywords_clean)

ApifyApiError: Authentication token was not provided

In [ ]:
linkedin_results

In [ ]:
# Fetch Naukri jobs using Apify
def fetch_naukri_jobs(search_query, location="Sri Lanka", rows=60):
    run_input = {
        'keyword': search_query,
        'maxJobs': 60,
        'freshness': 'all',
        'sortBy': 'relevance',
        'experience': 'all'
    }
    run = apify_client.actor('api_key_for_apify').call(run_input=run_input)
    jobs = list(apify_client.dataset(run['defaultDatasetId']).iterate_items())
    return jobs

#### Testing

In [ ]:
LINKEDIN_APIFY_ACTOR = os.getenv('LINKEDIN_APIFY_ACTOR')
NAUKRI_APIFY_ACTOR = os.getenv('NAUKRI_APIFY_ACTOR')

print(LINKEDIN_APIFY_ACTOR)
print(NAUKRI_APIFY_ACTOR)

umUj9ttmMtFzlhTa3
alpcnRV9YI9lYVPWk
